In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from fuzzymatcher import link_table, fuzzy_left_join
warnings.filterwarnings('ignore')

In [ ]:
# https://stackoverflow.com/questions/13636848/is-it-possible-to-do-fuzzy-match-merge-with-python-pandas

# This website lists out many functions/packages similar to fuzzyjoin in R


In [6]:
unique_addresses = pd.DataFrame(np.array([[1, 2], [4, 5], [7, 8]]),
                   columns=['a', 'b'])
cleaned = np.repeat(np.nan, len(unique_addresses))
cleaned[2] = 1

In [16]:
type(cleaned)

numpy.ndarray

In [ ]:
#02_Street_Matching_MNBK.R: [function] does address matching using Street Dictionary and filling down.

def street_match(sample, edict):
    """
    inputs:
    -------
    'sample': Census sample to match. Note: this is currently tailored to samples that have same columns as the original 100k sample. If column names change, adapt the code to clean column names accordingly, ensuring that relevant column names retain the names specified in the function.
    'edict': Street Dictionary used for matching. Ensure that each ED is a row and streets in the ED are in columns.
    
    outputs:
    -------
    The full, original census sample data input into the function with 3 additional columns: 
    best_match: Column containing best street name match
    result_type: 1 of 6 possible match types: (1) Perfect Match, (2) Identical Match, (3) Singular Mode, (4) Multiple Modes, (5) NA mode, (6) No match. Refer to documentation for more details.
    flag_st: Flag if street name match was ultimately obtained via fill down (which is the case for result type 4-6).
    The original street name is stored in \code{street_add}
    
    """
    
    # ---- FUNCTION ----
    
    ## clean column names: adapt code if sample has different columns. At minimum, ensure `record`, `ED` and `street_add` columns exist after cleaning.
    sample.columns = ["record", "township", "county", "ED", "person_id",
                     "dwelling_seq", "dwelling_ser", "dwelling_ser2", "hh_seq", "hh_seq_8",
                     "hh_ser2", "hh_ser_bef_split", "indiv_seq", "split", 
                     "line_no", "line_no2", "microfilm",
                     "n_fam", "n_person_bef_split", "house_num", "street_add"]
    
    ## clean ED number
    edict.ED = edict.ED.rjust(4, "0")
    
    ## Extract Unique Addresses
    unique_addresses = sample[sample.street_add.notna()][['ED','street_add']].unique()
    
    # ---- CLEAN ADDRESSES ----
    ## import rem_dup_word - remove duplicate words
    def rem_dup_word(x):
        x = x.lower()
        new_x = ' '.join(np.unique(x.split().strip()))
        return new_x
    
    ## [NOTE] make sure to run clean() function from '01_Street_Clean_Function_MNBK.R'
    
    ## preallocate memory for cleaned column 
    # cleaned <- rep(NA_character_, nrow(unique_addresses))
    cleaned = np.repeat(np.nan, len(unique_addresses))
    
    ## apply rem_dup_word() and str_clean()
    for i in 1:len(unique_addresses):
#         cleaned[i] = clean(unique_addresses.iloc[i, 2].apply(lambda x: rem_dup_word(str(x))).upper())
        
        cleaned[i] = unique_addresses.iloc[i, 2].apply(lambda x: rem_dup_word(str(x))).upper() 
        cleaned[i] = clean(cleaned[i])
    
    # ---- FIRST MATCH ----
    ## create add_matches, which combines `unique_addresses` with the `cleaned` column
    add_matches = unique_addresses[['ED', 'street_add']]
    add_matches['cleaned'] = cleaned
      
    ## create str_algo, which has all the methods we will use
    str_algo = ["dl", "qgram", "cosine", "jaccard", "jw"]

    ## create empty lists which will store outputs
    algo_list = list()
    ed_list = list()
    
    for i in add_matches.ED.unique():
        # create subset of edict and add_matches for a particular ED
        # TO ACCEPT BK: ED_dict <- filter(edict, ED == i & MNBK == "BK") %>%
        ED_dict = edict[edict['ED'] == i].dropna()
        ED_add = add_matches[add_matches['ED'] == i]
        
        for j in str_algo:
            # stringdist_join, extract best match for each method
            result = ED_add.merge(ED_dict, how = 'left', on = 'clean')
            result = result[['ED', 'raw', 'clean.x', 'clean.y', 'dscore']].groupby(by = 'raw').sort_values(by='dscore').iloc[0,:]
            
        # at this point, result = dataframe of addresses in ED (i), merged with method (j)
        # append result to algo_list, each element in algo_list is a dataframe
        algo_list[[j]] = result

        # "column bind" all elements in algo_list to form a df called ed_df
        # ed_df now contains all addresses in ED(i) with matches using all methods
        
        ed_df = algo_list[["ED", "raw", "clean.x"]]

        ed_list[i] = ed_df
        
    ## bind all dfs by row
    match_dict = ed_list[['ED', 'raw']]
    match_dict['clean'] = clean.x
    match_dict['match_dl'] = clean.y.x
    match_dict['match_qgram'] = clean.y.y
    match_dict['match_cos'] = clean.y.x.x
    match_dict['match_jac'] = clean.y.y.y
    match_dict['match_jw'] = clean.y
    match_dict['dscore_dl'] = dscore.x
    match_dict['dscore_qgram'] = dscore.y
    match_dict['dscore_cos'] = dscore.x.x
    match_dict['dscore_jac'] = dscore.y.y
    match_dict['dscore_jw'] = dscore
    
#     match_dict <- bind_rows(ed_list) %>%
#     select(ED = ED, raw = raw, clean = clean.x,
#            match_dl = clean.y.x, dscore_dl = dscore.x,
#            match_qgram = clean.y.y, dscore_qgram = dscore.y,
#            match_cos = clean.y.x.x, dscore_cos = dscore.x.x,
#            match_jac = clean.y.y.y, dscore_jac = dscore.y.y,
#            match_jw = clean.y, dscore_jw = dscore)
    
    # ---- FURTHER TUNING ----
    # create summary statistics
    
    summ_dscores = match_dict[['match_dl', 'match_qgram', 'match_cos', 'match_jac', 'match_jw', 'dscore_dl', 'dscore_qgram', 'dscore_cos', 'dscore_jac', 'dscore_jw']].fillna('5')
    summ_dscores.groupby('ED').agg({'dscore_dl':['mean', 'std'],
                                    'dscore_qgram':['mean', 'std'], 
                                    'dscore_cos':['mean', 'std'], 
                                    'dscore_jac':['mean', 'std']}).fillna('0').reset_index(inplace = True)
    
#     summ_dscores <- match_dict %>%
#     mutate(match_dl = ifelse(is.na(match_dl), "5", match_dl),
#            match_qgram = ifelse(is.na(match_qgram), "5", match_qgram),
#            match_cos = ifelse(is.na(match_cos), "5", match_cos),
#            match_jac = ifelse(is.na(match_jac), "5", match_jac),
#            match_jw = ifelse(is.na(match_jw), "5", match_jw),
#            dscore_dl = ifelse(is.na(dscore_dl), 5, dscore_dl),
#            dscore_qgram = ifelse(is.na(dscore_qgram), 5, dscore_qgram),
#            dscore_cos = ifelse(is.na(dscore_cos), 5, dscore_cos),
#            dscore_jac = ifelse(is.na(dscore_jac), 5, dscore_jac),
#            dscore_jw = ifelse(is.na(dscore_jw), 5, dscore_jw)) %>%
#     group_by(ED) %>%
#     summarize(mean_dl = mean(dscore_dl), sd_dl = sd(dscore_dl),
#               mean_qgram = mean(dscore_qgram), sd_qgram = sd(dscore_qgram),
#               mean_cos = mean(dscore_cos), sd_cos = sd(dscore_cos),
#               mean_jac = mean(dscore_jac), sd_jac = sd(dscore_jac),
#               mean_jw = mean(dscore_jw), sd_jw = sd(dscore_jw)) %>%
#     replace(is.na(.), 0)
    
    # format into a list such that e.g. `summ_dscores_list$0010$jw$mean` gives mean jw dscore of ED 0010
    summ_dscores_list = list(summ_dscores.ED.unique())
    
#     summ_dscores_list <- unique(summ_dscores$ED) %>% as.list()
#     names(summ_dscores_list) <- unique(summ_dscores$ED)

    for i in summ_dscores_list:
        s = summ_dscores[summ_dscores['ED'] == i]
        ED_sum_list = list({'dl': s.mean_dl, 'sd':s.sd_dl}, {'qgram': s.mean_qgram})
        
        summ_dscores_list[[i]] = ED_sum_list
#     for (i in names(summ_dscores_list)) {
#     s <- filter(summ_dscores, ED == i)
#     ED_sum_list <- list(dl = list(mean = s$mean_dl, sd = s$sd_dl),
#                         qgram = list(mean = s$mean_qgram, sd = s$sd_qgram),
#                         cosine = list(mean = s$mean_cos, sd = s$sd_cos),
#                         jaccard = list(mean = s$mean_jac, sd = s$sd_jac),
#                         jw = list(mean = s$mean_jw, sd = s$sd_jw))
#     summ_dscores_list[[i]] <- ED_sum_list
#   }
    
    
    

    # match again
    for i in add_matches.ED.unique():
        ED_dict = edict[edict['ED'] == i]
        
        ED_add = add_matches[add_matches['ED'] == i]
        
        for j in str_algo:
            threshold = summ_dscores_list[[i][j]].mean + 2*summ_dscores_list[[i]][[j]].sd
            result = ED_add.merge(ED_dict, how = 'left', on = 'clean')
            result = result[['ED', 'raw', 'clean.x', 'clean.y', 'dscore']].groupby(by = 'raw')
            
            algo_list[[j]] = result
            ef_df = algo_list
        
        ed_list[[i]] = ed_df
    
#     for (i in unique(add_matches$ED)) {
#     # create subset of edict and add_matches for a particular ED
#     # TO ACCEPT BK: ED_dict <- filter(edict, ED == i & MNBK == "BK") %>%
#     ED_dict <- filter(edict, ED == i) %>% 
#       select(- "ED") %>%
#       unlist() 
#     ED_dict <- data.frame(clean = ED_dict, stringsAsFactors = FALSE) %>%
#       filter(!is.na(clean))
#     ED_add <- filter(add_matches, ED == i)
    
#     for (j in str_algo) {
#       # set a max_dist that varies by ED. max_dist is 2sd higher than mean.
#       threshold <- summ_dscores_list[[i]][[j]]$mean + 2 * summ_dscores_list[[i]][[j]]$sd
      
#       # stringdist_join, extract best match for each method
#       result <- stringdist_left_join(ED_add, ED_dict, by = "clean", 
#                                      max_dist = threshold, method = j, distance_col = "dscore") %>%
#         select(ED, raw, clean.x, clean.y, dscore) %>%
#         group_by(raw) %>%
#         arrange(dscore) %>%
#         slice(1)
      
#       # at this point, result = dataframe of addresses in ED (i), merged with method (j)
#       # append result to algo_list, each element in algo_list is a dataframe
#       algo_list[[j]] <- result
      
#       # "column bind" all elements in algo_list to form a df called ed_df
#       # ed_df now contains all addresses in ED(i) with matches using all methods
#       ed_df <- algo_list %>% 
#         reduce(left_join, by = c("ED", "raw", "clean.x"))
#     }
#     # append each ed_df to ed_list 
#     ed_list[[i]] <- ed_df
#   }


    ## bind all dfs by row
    match_dict = ed_list[['ED', 'raw']]
    match_dict['clean'] = clean.x
    match_dict['match_dl'] = clean.y.x
    match_dict['match_qgram'] = clean.y.y
    match_dict['match_cos'] = clean.y.x.x
    match_dict['match_jac'] = clean.y.y.y
    match_dict['match_jw'] = clean.y
    match_dict['dscore_dl'] = dscore.x
    match_dict['dscore_qgram'] = dscore.y
    match_dict['dscore_cos'] = dscore.x.x
    match_dict['dscore_jac'] = dscore.y.y
    match_dict['dscore_jw'] = dscore
    
#     match_dict <- bind_rows(ed_list) %>%
#     select(ED = ED, raw = raw, clean = clean.x,
#            match_dl = clean.y.x, dscore_dl = dscore.x,
#            match_qgram = clean.y.y, dscore_qgram = dscore.y,
#            match_cos = clean.y.x.x, dscore_cos = dscore.x.x,
#            match_jac = clean.y.y.y, dscore_jac = dscore.y.y,
#            match_jw = clean.y, dscore_jw = dscore)
    
    # ---- BEST MATCH ----
    # function to extract modes
    
    def modal(x):
        freq = pd.DataFrame(x.size())
        result = 
        
        if len(result) > 1:
            result = "+".join(result)
        
        return result
       
#     modal <- function(x) {
#     freq <- table(x) %>% as.data.frame(stringsAsFactors = FALSE)
#     result <- freq[which(freq$Freq == max(freq$Freq)), 1]
#     if (length(result) > 1){
#       result <- str_c(result, collapse = "+")
#     }
#     result
#   }
    
    # extract modes
    match_dict[['match_dl', 'match_qgram', 'match_cos', 'match_jac', 'match_jw', 'dscore_dl', 'dscore_qgram', 'dscore_cos', 'dscore_jac', 'dscore_jw']].fillna('0')
    match_dict_modes = match_dict[['ED', 'raw', 'clean', 'match_dl', 'match_qgram', 'match_cos', 'match_jac', 'match_jw']]
    
    
#     match_dict_modes <- match_dict %>%
#     mutate(match_dl = ifelse(is.na(match_dl), "0", match_dl),
#            match_qgram = ifelse(is.na(match_qgram), "0", match_qgram),
#            match_cos = ifelse(is.na(match_cos), "0", match_cos),
#            match_jac = ifelse(is.na(match_jac), "0", match_jac),
#            match_jw = ifelse(is.na(match_jw), "0", match_jw),
#            dscore_dl = ifelse(is.na(dscore_dl), 0, dscore_dl),
#            dscore_qgram = ifelse(is.na(dscore_qgram), 0, dscore_qgram),
#            dscore_cos = ifelse(is.na(dscore_cos), 0, dscore_cos),
#            dscore_jac = ifelse(is.na(dscore_jac), 0, dscore_jac),
#            dscore_jw = ifelse(is.na(dscore_jw), 0, dscore_jw)) %>%
#     select(ED, raw, clean, match_dl, match_qgram, 
#            match_cos, match_jac, match_jw) %>%
#     gather("method", "match", - c("ED", "raw", "clean")) %>%
#     group_by(ED, raw) %>%
#     summarise(mode = modal(match))


    # join to match_dict
    match_dict = pd.merge(match_dict, match_dict_modes, how = 'left', on =['ED', 'raw'])
#     match_dict <- left_join(match_dict, match_dict_modes, by = c("ED", "raw"))

    # best match
    match_dict[['match_dl', 'match_qgram', 'match_cos', 'match_jac', 'match_jw', 'dscore_dl', 'dscore_qgram', 'dscore_cos', 'dscore_jac', 'dscore_jw']].fillna('0')
           
#     match_dict <- match_dict %>%
#     mutate(match_dl = ifelse(is.na(match_dl), "0", match_dl),
#            match_qgram = ifelse(is.na(match_qgram), "0", match_qgram),
#            match_cos = ifelse(is.na(match_cos), "0", match_cos),
#            match_jac = ifelse(is.na(match_jac), "0", match_jac),
#            match_jw = ifelse(is.na(match_jw), "0", match_jw),
#            dscore_dl = ifelse(is.na(dscore_dl), 0, dscore_dl),
#            dscore_qgram = ifelse(is.na(dscore_qgram), 0, dscore_qgram),
#            dscore_cos = ifelse(is.na(dscore_cos), 0, dscore_cos),
#            dscore_jac = ifelse(is.na(dscore_jac), 0, dscore_jac),
#            dscore_jw = ifelse(is.na(dscore_jw), 0, dscore_jw)) %>%
#     rowwise() %>%
#     mutate(result_type = ifelse(match_dl == match_cos & match_dl == match_qgram & match_dl == match_jac & match_dl == match_jw, ifelse(clean == match_dl, 1, ifelse(mode == 0, 6, 2)), ifelse(str_detect(mode, "\\+"), 4, ifelse(mode == 0, 5, 3)))) %>%
#     mutate(best_match = case_when(result_type %in% c(1, 2, 3) ~ mode,
#                                   result_type %in% c(4, 5, 6) ~ "0"))
    
    
    # ---- FILL DOWN ----
    # create flags for row that had to be filled
    match_dict = np.where(match_dict.best_match == 0, 1, 0)
    match_dict
    na_matches = match_dict[match_dict['best_match'] == 0]
    
#     match_dict <- mutate(match_dict, flag = ifelse(best_match == 0, 1, 0))
#     match_dict$index <- 1:nrow(match_dict)
  
#     na_matches <- filter(match_dict, best_match == 0)


    # fill down
    
    for i in na_matches.index:
        
        match_ED = na_matches[na_matches.index == i]['ED']
        
        
    
# for (i in na_matches$index) {
#     # extract pool of potential matches
#     i <- as.numeric(i)
#     match_ED <- na_matches[na_matches$index == i, "ED"]
#     pool_indexes <- c(i - 3, i - 2, i - 1, i + 1, i + 2, i + 3)
#     pool_matches <- filter(match_dict, index %in% pool_indexes & ED == match_ED) %>%
#       select(best_match) %>%
#       distinct()
    
        # match using jw
        match = 
        
#         match <- stringdist_left_join(filter(na_matches, index == i), pool_matches, by = c(clean = "best_match"), 
#                                       max_dist = 5, method = "jw", distance_col = "dscore") %>%
#           group_by(raw) %>%
#           arrange(dscore) %>%
#           slice(1)
    
    # sub match into match_dict
    match_dict[match_dict.index == i]['best_match'] = match['best_match.y']
#     match_dict[match_dict$index == i, "best_match"] <- match$best_match.y
#   }

    # ---- MERGE TO ORIGINAL SAMPLE ----
    match_dict_subset = match_dict[[]]
    sample_cleaned = pd.merge(sample, match_dict_subset, how = 'left', left_on = ["ED", "ED"], right_on = ["street_add", "raw"])
    
#   match_dict_subset <- select(match_dict, ED, raw, best_match, result_type, flag_st = flag)
#   sample_cleaned <- left_join(sample, match_dict_subset, by = c("ED" = "ED", "street_add" = "raw"))
# }
    
    